In [40]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob 
import os

In [ ]:
benign = "./dataset_benign.csv"
malware = "./dataset_malware.csv"

In [41]:
files = ["./dataset_benign.csv", "./dataset_malware.csv"]

data = pd.concat(map(pd.read_csv, files), ignore_index=True)

data = data.sample(frac=1, ignore_index=True)

data

# data.to_csv('./dataset.csv')

,Name,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,...,SectionMaxChar,SectionMainChar,DirectoryEntryImport,DirectoryEntryImportSize,DirectoryEntryExport,ImageDirectoryEntryExport,ImageDirectoryEntryImport,ImageDirectoryEntryResource,ImageDirectoryEntryException,ImageDirectoryEntrySecurity
0,VirusShare_134af908be9ee2f55da8f03aab28cdda,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,7,179,0,0,40960,45056,0,0
1,VirusShare_c398fb635b66566c88d5308714910758,23117,144,3,0,4,0,65535,0,184,...,3758096512,0,1,3,4,41460,46062,0,0,0
2,VirusShare_3e87de351cc839fac310847b1e8d3bfa,23117,80,2,0,4,15,65535,0,184,...,3221225536,0,13,378,0,0,450560,507904,0,0
3,VirusShare_3de66f183b2d6f4038065afcd61b7d37,23117,144,3,0,4,0,65535,0,184,...,3758096448,0,2,43,0,0,21968,24576,0,0
4,VirusShare_6ce1e1060ef2dbe4554c0404a887c422,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,2,88,5,601136,599212,622592,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91170,VirusShare_2162d79cbc7030eb4d95dbef5b31d4c2,23117,144,3,0,4,0,65535,0,184,...,3758096480,0,1,4,0,0,36996,36864,0,0
91171,edpauditapi.dll,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,28,151,13,121440,121936,151552,139264,0
91172,VirusShare_771586cb072a4079e57602b387791ac6,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,4,106,4,124816,122360,0,0,0
91173,WSReset.exe,23117,144,3,0,4,0,65535,0,184,...,3221225536,0,8,73,0,0,21968,32768,28672,0


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91175 entries, 0 to 91174
Data columns (total 78 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Name                          91175 non-null  object 
 1   e_magic                       91175 non-null  int64  
 2   e_cblp                        91175 non-null  int64  
 3   e_cp                          91175 non-null  int64  
 4   e_crlc                        91175 non-null  int64  
 5   e_cparhdr                     91175 non-null  int64  
 6   e_minalloc                    91175 non-null  int64  
 7   e_maxalloc                    91175 non-null  int64  
 8   e_ss                          91175 non-null  int64  
 9   e_sp                          91175 non-null  int64  
 10  e_csum                        91175 non-null  int64  
 11  e_ip                          91175 non-null  int64  
 12  e_cs                          91175 non-null  int64  
 13  e